In [202]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import geopandas
data = pd.read_csv("https://data.wprdc.org/datastore/dump/fbb50b02-2879-47cd-abea-ae697ec05170")
df = pd.read_csv("Pittsburgh Education 2024.csv")

neighborhoods = geopandas.read_file("Neighborhoods/Neighborhoods_.shp")

In [275]:
#DATA SET 1

# Jack A
facilities = data["neighborhood"].value_counts().sort_values(ascending=False) 
facilities_weighted = 0.25*facilities / 412
print(facilities.head())
#facilities_query_mask = facilities_weighted > 0.02

# Jack B
least_important = ["OFFICE", "Office", "Salt Dome", "TRAINING", "Training", "Pool Closed", "Vacant", "Storage", "STORAGE", 
                   "SERVICE", "Service", "Concession", "Dugout", "Shelter", "Cabin", "Restrooms", "Utility"]
data_filtered = data[~data["type"].isin(least_important)]
data_filtered_grouped = data_filtered["neighborhood"].value_counts().sort_values(ascending=False)
important_facilities = 0.75 * (data_filtered_grouped / 132)

# lenny
school_counts = df['Neighborhood'].value_counts()
valid_neighborhoods = school_counts[school_counts >= 2].index
stability = df[df['Neighborhood'].isin(valid_neighborhoods)].groupby('Neighborhood')['Student Stability Rate'].mean()
approval = df[df['Neighborhood'].isin(valid_neighborhoods)].groupby('Neighborhood')['Teacher Approval Rate'].mean()
ratio_inverse = 100 - df[df['Neighborhood'].isin(valid_neighborhoods)].groupby('Neighborhood')['Student/Teacher Ratio'].mean()
combined = pd.concat([stability, approval, ratio_inverse], axis=1)
combined.columns = ['Stability', 'Approval', '100 - Ratio']
combined['Best Education Score'] = combined.mean(axis=1).sort_index()
bars = combined['Best Education Score'].sort_index().plot(kind='bar', color='orange', edgecolor='black')
combined['Best Education Score'].sort_values().cumsum()
# total score:  1041.846150
combined['Weighted Education Score'] = combined['Best Education Score'] / 1041.846150
education_weighted = combined.drop(['Best Education Score', 'Stability','Approval','100 - Ratio'],axis=1)
education_weighted = education_weighted.reset_index()# = 'neighborhood'
education_weighted = education_weighted.rename(columns={'Neighborhood': 'neighborhood'})
education_weighted.head()
# Nate

neighborhood
Highland Park          30
Squirrel Hill South    29
Perry North            20
Brookline              19
Strip District         15
Name: count, dtype: int64


,neighborhood,Weighted Education Score
0,Allegheny Center,0.082394
1,Beechview,0.083380
2,Brookline,0.083826
3,Carrick,0.085149
4,Downtown,0.082112


In [277]:
# for normal bar graph
data_concat = pd.concat([important_facilities,facilities_weighted,education_weighted])

#data_merged = pd.merge(data_concat,education_weighted, left_on="neighborhood",right_on="Neighborhood")
print(data_concat.head())
#data_combined = data_concat.groupby('neighborhood').sum().sort_values(ascending=False)
data_combined_mask = data_combined > 0.03
data_combined[data_combined_mask].plot.bar()

# for stacked bar graph 
merged = pd.merge(important_facilities, facilities_weighted, on='neighborhood', how='outer', suffixes=('_important', '_normal')).fillna(0)
merged['total'] = merged['count_important'] + merged['count_normal']
merged = merged2.sort_values('total', ascending=False)
merged_graphthis = merged.drop('total', axis=1)
merged_graphthis[data_combined_mask].plot(
    kind='bar',
    stacked=True,
    ylabel='Count',
    title='Counts by Source (Stacked)',
)
#plt.show()
# .46 + .37

                             count neighborhood  Weighted Education Score
Lincoln-Lemington-Belmar  0.045455          NaN                       NaN
Brookline                 0.028409          NaN                       NaN
Hazelwood                 0.028409          NaN                       NaN
Brighton Heights          0.022727          NaN                       NaN
South Side Flats          0.022727          NaN                       NaN


<Axes: title={'center': 'Counts by Source (Stacked)'}, xlabel='neighborhood', ylabel='Count'>

In [19]:
facilities = data["neighborhood"].value_counts().sort_values(ascending=False) 
facilities_query_mask = facilities > 10
facilities[facilities_query_mask].plot.bar()
#plt.show()

<Axes: xlabel='neighborhood'>

In [20]:
least_important = ["OFFICE", "Office", "Salt Dome", "TRAINING", "Training", "Pool Closed", "Vacant", "Storage", "STORAGE", 
                   "SERVICE", "Service", "Concession", "Dugout", "Shelter", "Cabin", "Restrooms", "Utility"]
data_filtered = data[~data["type"].isin(least_important)]
data_filtered_grouped = data_filtered["neighborhood"].value_counts().sort_values(ascending=False)
amt_query_mask_filtered = data_filtered_grouped > 3
data_filtered_grouped[amt_query_mask_filtered].plot.bar()
#plt.show()

# edit axis, labels, etc

<Axes: xlabel='neighborhood'>

In [ ]:
# Some code borrowed from geopandas example in lab lesson

facilities.name = "facAmt"
facilities_map = neighborhoods.merge(facilities, how='left', left_on='hood', right_on='neighborhood')
pd.set_option('display.max_columns', None)
facilities_map.plot(column='facAmt', # set the data to be used for coloring
               cmap="Greens",              # choose a color palette
               edgecolor="white",        # outline the districts in white
               legend=True,              # show the legend
               legend_kwds={'label': "Number of Facilities (unfiltered)"}, # label the legend
               figsize=(15, 10),         # set the size
               missing_kwds={"color": "gray"} # set disctricts with no data to gray
               )

data_filtered_grouped.name = "DFG"
facilities_map = neighborhoods.merge(data_filtered_grouped, how='left', left_on='hood', right_on='neighborhood')
pd.set_option('display.max_columns', None)
facilities_map.plot(column='DFG', # set the data to be used for coloring
               cmap="Reds",              # choose a color palette
               edgecolor="white",        # outline the districts in white
               legend=True,              # show the legend
               legend_kwds={'label': "Number of Facilities (filtered twice)"}, # label the legend
               figsize=(15, 10),         # set the size
               missing_kwds={"color": "gray"} # set disctricts with no data to gray
               )
plt.show()

